<a href="https://colab.research.google.com/github/kerotan7-kaeru/MediaPipe/blob/main/MediaPipe_Hands_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google MediaPipe Hands
#
# 参考：
# 　・Hand landmarks detection guide | Google AI Edge - Gemini API
#     https://ai.google.dev/edge/mediapipe/solutions/vision/hand_landmarker?hl=ja
# 　・MediaPipe 入門 (3) - Hands｜npaka
#     https://note.com/npaka/n/nbe59c36b3d49　←🐸今回のCodeはここから持ってきた

In [ ]:
# 🐸追加
# Google Driveを接続する為のおまじないコード

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 🐸追加
# 仮想コンピュータ(Colab)に、Google MediaPipeをインストール

!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [ ]:
import cv2
import mediapipe as mp

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [ ]:
# 🐸追加

# フォルダのパスを指定
# 　./drive/MyDriveの下に、/Colab Notebooks/yuremiruzu フォルダを作っておく
folder_path = './drive/MyDrive/Colab Notebooks/yuremiruzu/'

# データファイルを指定
file_list = [folder_path + '米田の手のひら_01.jpg',
             folder_path + '有末先生の手のひら_01.png']  # 画像ファイルを幾つでも

# アップロードした画像ファイルのリスト
# file_list = ['/content/drive/MyDrive/Colab Notebooks/yuremiruzu/米田の手のひら_01.jpg',
#              '/content/drive/MyDrive/Colab Notebooks/yuremiruzu/有末先生の手のひら_01.png']  # ここに画像ファイルのパスを追加

In [ ]:
# 静止画像の場合：
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx, file in enumerate(file_list):
    # 画像を読み取り、利き手が正しく出力されるようにy軸を中心に反転
    image = cv2.flip(cv2.imread(file), 1)
    # 処理する前にBGR画像をRGBに変換
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # 利き手を出力し、画像に手のランドマークを描画
    print('Handedness:', results.multi_handedness)
    if not results.multi_hand_landmarks:
      continue
    image_height, image_width, _ = image.shape
    annotated_image = image.copy()
    for hand_landmarks in results.multi_hand_landmarks:
      print('hand_landmarks:', hand_landmarks)
      print(
          f'Index finger tip coordinates: (',
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
      )
      mp_drawing.draw_landmarks(
          annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    cv2.imwrite(
        # '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))  # 元のcode
        # 🐸修正したcode
        folder_path + str(idx) + '.png', cv2.flip(annotated_image, 1))

# cap.release()

Handedness: [classification {
  index: 0
  score: 0.964780211
  label: "Left"
}
]
hand_landmarks: landmark {
  x: 0.399308175
  y: 0.852176
  z: 7.31967305e-07
}
landmark {
  x: 0.508947372
  y: 0.865660071
  z: -0.0405338891
}
landmark {
  x: 0.60683465
  y: 0.823126793
  z: -0.0534962527
}
landmark {
  x: 0.69095695
  y: 0.780032814
  z: -0.0650108
}
landmark {
  x: 0.756046176
  y: 0.770626843
  z: -0.0751570389
}
landmark {
  x: 0.591253877
  y: 0.63177228
  z: 0.000589285453
}
landmark {
  x: 0.651601374
  y: 0.537653923
  z: -0.00631692214
}
landmark {
  x: 0.688978
  y: 0.475472212
  z: -0.016820481
}
landmark {
  x: 0.716019273
  y: 0.424297512
  z: -0.0269936621
}
landmark {
  x: 0.541225255
  y: 0.584235489
  z: 0.00372446142
}
landmark {
  x: 0.585973561
  y: 0.466255665
  z: 0.00125418743
}
landmark {
  x: 0.611494
  y: 0.392828584
  z: -0.00892013405
}
landmark {
  x: 0.63208425
  y: 0.326620281
  z: -0.0176086891
}
landmark {
  x: 0.48904264
  y: 0.561059117
  z: -0.00075

In [ ]:
# 🐸　以下はこれから

In [ ]:
# Webカメラ入力の場合：
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # ビデオをロードする場合は、「continue」ではなく「break」を使用してください
      continue

    # 後で自分撮りビューを表示するために画像を水平方向に反転し、BGR画像をRGBに変換
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # パフォーマンスを向上させるために、オプションで、参照渡しのためにイメージを書き込み不可としてマーク
    image.flags.writeable = False
    results = hands.process(image)

    # 画像に手のアノテーションを描画
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    cv2.imshow('MediaPipe Hands', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()